#### IMPORTAMOS LIBRERIAS A USAR

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

## Cargamos los datos a utilizar
Preview de los datos

In [2]:
data = pd.read_csv('../datasets/data_credit.csv')
datano = data[data['Approved']==0]
datasi = data[data['Approved']==1]
datano.head()

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
70,1,32.33,7.5,1,1,Education,Asian,1.585,1,0,0,1,ByOtherMeans,420,0,0
71,1,34.83,4.0,1,1,Real Estate,Asian,12.500,1,0,0,1,ByBirth,0,0,0
72,0,38.58,5.0,1,1,InformationTechnology,White,13.500,1,0,0,1,ByBirth,980,0,0
73,1,44.25,0.5,1,1,CommunicationServices,White,10.750,1,0,0,0,ByOtherMeans,400,0,0
74,1,44.83,7.0,0,0,Energy,White,1.625,0,0,0,0,ByBirth,160,2,0


### PROCESSING DATA

In [3]:
data = data.drop(['Industry','Citizen','Ethnicity'], axis=1)

### TRAIN AND TEST SET

In [4]:
split_ratio = 0.80 # split data 60
split_index = int(split_ratio*len(data))
Y_train = np.array([data["Approved"][:split_index]])
Y_test = np.array([data["Approved"][split_index:]])
X_train = np.array(data[:split_index].drop(columns="Approved").T)
X_test = np.array(data[split_index:].drop(columns="Approved").T)
print(X_test.shape, Y_test.shape)

(12, 138) (1, 138)


In [5]:
def layer_sizes(X,Y):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    n_h = 8

    return (n_x,n_y,n_h)

In [6]:
print(layer_sizes(X_train, Y_train))

(12, 1, 8)


## Inicializamos los parámetros de la red neuronal


In [7]:
def parameter_initialization(n_h, n_x, n_y):
    
    # Donde n_h será el tamaño de la capa oculta
    # n_x Tamaño de la capa de entrada
    # n_y Tamaño de la capa de salida


    np.random.seed(1)

    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y,1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2":b2}
    
    return parameters
    

In [8]:
parameters = (parameter_initialization(2,2,1))
print("W1: \n",parameters["W1"])
print("B1: \n",parameters["b1"])
print("W2: \n",parameters["W2"])
print("B2: \n",parameters["b2"])

W1: 
 [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
B1: 
 [[0.]
 [0.]]
W2: 
 [[ 0.00865408 -0.02301539]]
B2: 
 [[0.]]


### FORWARD PROPAGATION

#### ACTIVATION FUNCTIONS

In [9]:
def relu(x):
    return np.maximum(x,0)
    

In [10]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### FORWARD PROPAGATION

In [11]:
def forward_p(X, parameters):

    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1)+b2
    A2 = sigmoid(Z2)

    print(X.shape, A2.shape)
    assert(A2.shape == (1,X.shape[1]))
    
    cache = {"Z1": Z1,
             "Z2": Z2,
             "A1": A1,
             "A2": A2 }
    
    return A2, cache
    
    

### COST FUNCTION

In [12]:
def cost(A2, Y):##
    m = Y.shape[0]
    print("m:", m)
    logprobs = np.multiply(np.log(A2),Y)+np.multiply((1-Y),np.log(1-A2)) 
    cost = - np.sum(logprobs)/m
    cost = float(np.squeeze(cost))
    return cost

### BACKWARD PROPAGATION

In [13]:
def backward_p(parameters,cache, X,Y):##
    m = X.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    A1 = cache["A1"]
    A2 = cache["A2"]

    dZ2 = A2-Y
    dW2 = np.dot(dZ2,A1.T)/m
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m
    dZ1 = np.multiply(np.dot(W2.T,dZ2),1 - np.power(A1, 2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1,axis=1, keepdims=True)/m

    grads = {
            "dW1":dW1,
            "db1":db1,
            "dW2":dW2,
            "db2":db2
            }

    
    return grads

### GRADIENT DESCENT TO UPDATE PARAMETERS

In [14]:
def gd_update_parameters(parameters, grads, learning_rate=1.2): ## 
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2

    parameters = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }
        
    return parameters

## Neural network implementation function

In [15]:
def model_imp(X,Y, n_h, num_iterations = 2000, print_cost=False):
    np.random.seed(3)
    n_x = layer_sizes(X,Y)[0]
    n_y = layer_sizes(X,Y)[1]
    parameters = parameter_initialization(n_h,n_x,n_y)
    for i in range(0, num_iterations):
        A2, cache = forward_p(X, parameters)
        cost_c = cost(A2,Y)
        grads = backward_p(parameters, cache, X, Y)
        parameters = gd_update_parameters(parameters, grads)
        if print_cost and i % 1000 == 0:
            print("COST ITERATION %i: %f" %(i,cost_c)) ## Se estaba usando la función costo, en vez de variable
    return parameters
    

In [16]:
def predict(parameters, X):
    A2,cache = forward_p(X,parameters)
    predictions = np.round(A2)
    return predictions

### TEST MODEL

In [17]:
parameters = model_imp(X_train, Y_train, n_h=10, num_iterations=14000, print_cost=True)


(12, 552) (1, 552)
m: 1
COST ITERATION 0: 380.889200
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (1, 552)
m: 1
(12, 552) (

### PREDICTION WITH TRAINING DATA

In [18]:
predictions = predict(parameters, X_train)
print ("Accuray", accuracy_score(Y_train[0], predictions[0])*100)
print("Predictions: \n", predictions)

(12, 552) (1, 552)
Accuray 73.0072463768116
Predictions: 
 [[0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0.
  0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0.
  0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
  0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
  1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1.
  1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
  0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
  0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

### PREDICTION WITH TEST DATA

In [19]:
predictionsTest = predict(parameters, X_test)
print ("Accuray", accuracy_score(Y_test[0], predictionsTest[0])*100)
print("Predictions: \n", predictionsTest)

(12, 138) (1, 138)
Accuray 71.01449275362319
Predictions: 
 [[1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1.
  1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
  0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1.]]
